# Imports packages and assign variables

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
file_paths = [
    'testData/1.10_A_30s_600ms_3/',
    'testData/2.50_A_30s_600ms_3/',
    'testData/3.90_A_30s_600ms_3/',
    'testData/4.130_A_30s_600ms_3/',
    'testData/5.60_A_30s_200ms_3/',
    'testData/6.60_A_30s_500ms_3/',
    'testData/7.60_A_30s_800ms_3/',
    'testData/8.60_A_30s_1100ms_3/',
    'testData/9.60_A_5s_600ms_3/',
    'testData/10.60_A_20s_600ms_3/',
    'testData/11.60_A_35s_600ms_3/',
    'testData/12.60_A_50s_600ms_3/',
    'testData/13.60_A_30s_600ms_2/',
    'testData/14.60_A_30s_600ms_4/',
    'testData/15.60_A_30s_600ms_6/',
    'testData/16.60_A_30s_600ms_8/',
    'testData/17.60_A_30s_600ms_3/',
    'testData/18.60_A+NA+AZA_30s_600ms_3/',
    'testData/19.60_A+NA_30s_600ms_3/',
    'testData/20.60_A+AZA_30s_600ms_3/',
    ]

labels = [ _[12:-1] if _ [11] == '.' else _[11:-1] for _ in file_paths ]

# Get mean time for endponits

In [ ]:
output_path = 'testCharts/meanTime'
os.makedirs(output_path, exist_ok=True)
i = 1
for path, label in zip(file_paths, labels):
    marketlog = pd.read_csv(path + 'stockApp_marketlog.csv')
    trafficlog = pd.read_csv(path + 'stockApp_trafficlog.csv')
    merged_data = pd.merge(trafficlog,marketlog, on='request_id')
    merged_data = merged_data[~merged_data['endpoint_url'].isin(['/api/signIn', '/api/signUp', '/api/addCompany'])]
    average_times = merged_data.groupby('endpoint_url').agg({
        'api_time': 'mean',
        'application_time': 'mean',
        'database_time': 'mean',
        'request_id': 'count'
    }).reset_index()

    average_times.rename(columns={'request_id': 'count'}, inplace=True)
    average_times['endpoint_label'] = average_times.apply(lambda row: f"{row['endpoint_url'][4:]} \n (requests: {row['count']})", axis=1)
    average_times =  average_times.drop(columns=['count'])
    ax = average_times.plot(x='endpoint_label', kind='bar', figsize=(12, 8), width=0.8)
    for j in ax.patches:
        ax.annotate(f'{j.get_height():.4f}', 
                (j.get_x() + j.get_width() / 2, j.get_height()), 
                ha='center', va='bottom', fontsize=10)
    plt.xticks(rotation=0, ha='right', fontsize=8)
    ax.set_xticklabels(average_times['endpoint_label'], rotation=0, ha='center', fontsize=8, wrap=True)
    plt.xlabel('Endpoint URL')
    plt.ylabel('Mean time (s)')
    plt.title('Mean time | ' + label)
    plt.grid(True)
    plt.legend(title='Times')
    plt.tight_layout()
    file_name = f'{i}.{label}_plot.svg'
    #plt.savefig(os.path.join(output_path, file_name))
    plt.show()
    plt.close()
    i +=1

# Mean trade times for all companies in one cycle. Copmare test with the same changing parameter

In [ ]:
output_path = 'testCharts/meanTimeTrade'
os.makedirs(output_path, exist_ok=True)
group_size = [4,4,4,4,4]
i = 0
parametrs = ['user', 'request time', 'transaction time', 'number of transacion workers', 'user class','none',
             'user', 'request time', 'transaction time', 'number of transacion workers']
transactions_times = [30,30,30,30,30,30,30,30,5,20,35,50,30,30,30,30,30,30,30,30]
start_idx = 0
while start_idx < len(file_paths):
    files_to_process = file_paths[start_idx:start_idx + group_size[i]]
    labels_to_use = labels[start_idx:start_idx + group_size[i]]
    parametr = parametrs[i]
    plt.figure(figsize=(12, 8))
    avg_application_times = []
    avg_database_times = []
    transactions_times_to_use = transactions_times[start_idx:start_idx +group_size[i]]
    j = 0
    for path, label in zip(files_to_process, labels_to_use):
        tradelog = pd.read_csv(path + 'stockApp_tradelog.csv')
        if 'company_ids' not in tradelog.columns:
            print(f"Brak kolumny 'company_ids' w danych: {label}")
            continue
        grouped = tradelog.groupby('company_ids').agg({
            'application_time': 'mean',
            'database_time': 'mean'
        }).reset_index()
        avg_application_time = grouped['application_time'].sum() / transactions_times_to_use[j] * 100
        avg_database_time = grouped['database_time'].sum() / transactions_times_to_use[j] * 100
        avg_application_times.append(avg_application_time)
        avg_database_times.append(avg_database_time)
        j += 1
    
    x = range(len(files_to_process))
    bar1 = plt.bar([_ - 0.2 for _ in x], avg_application_times, width=0.4, label='Application Time')
    bar2 = plt.bar([_ + 0.2 for _ in x], avg_database_times, width=0.4, label='Database Time')
    for bar in bar1:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 2), ha='center', va='bottom')
    for bar in bar2:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 2), ha='center', va='bottom')
    plt.xticks(ticks=[_ for _ in range(group_size[i])], labels=labels_to_use) 
    plt.ylabel('Average Time (%)')
    plt.title(f'Mean transaction time | {parametr}')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    file_name = f'{parametr}_plot.svg'
    #plt.savefig(os.path.join(output_path, file_name))
    plt.show()
    plt.close()
    start_idx += group_size[i]
    i +=1

# Response time over time test. Copmare test with the same changing parameter

In [ ]:
output_path = 'testCharts/responseTime'
os.makedirs(output_path, exist_ok=True)
group_sizes = [4,4,4,4,4]
index = 0
parametrs = ['user', 'request time', 'transaction time', 'number of transacion workers', 'user class','none',
             'user', 'request time', 'transaction time', 'number of transacion workers']
for i, size in enumerate(group_sizes):
    files_to_process = file_paths[index:index + size]
    labels_to_use = labels[index:index + size]
    parametr = parametrs[i]
    plt.figure(figsize=(12, 8))
    for path, label in zip(files_to_process, labels_to_use):
        marketlog = pd.read_csv(path + 'stockApp_marketlog.csv')
        trafficlog = pd.read_csv(path + 'stockApp_trafficlog.csv')
        marketlog = marketlog.drop(columns='timestamp')
        marketlog = pd.merge(marketlog, trafficlog, on='request_id')
        marketlog = marketlog[~marketlog['endpoint_url'].isin(['/api/signIn', '/api/signUp', '/api/addCompany'])]
        marketlog['timestamp'] = pd.to_datetime(marketlog['timestamp'])
        marketlog = marketlog[['timestamp', 'api_time']] 
        start_time = marketlog['timestamp'].min()
        marketlog['relative_time'] = (marketlog['timestamp'] - start_time).dt.total_seconds()
        marketlog = marketlog.set_index('timestamp').resample('5s').mean().reset_index()
        marketlog = marketlog.dropna()
        plt.plot(marketlog['relative_time'], marketlog['api_time'], linestyle='-', label=label)
    plt.xlabel('Timestamp')
    plt.ylabel('API Time (s)')
    plt.title(f'API Time | {parametr}')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    file_name = f'{parametr}_api_time_plot.svg'
    plt.show()
    #plt.savefig(os.path.join(output_path, file_name))
    plt.close()
    index += size

# Trade time over time. Copmare test with the same changing parameter

In [ ]:
output_path = 'testCharts/tradeTime'
os.makedirs(output_path, exist_ok=True)
group_sizes = [4,4,4,4,4]
index = 0
parametrs = ['user', 'request time', 'transaction time', 'number of transacion workers', 'user class','none',
             'user', 'request time', 'transaction time', 'number of transacion workers']
for i, size in enumerate(group_sizes):
    files_to_process = file_paths[index:index + size]
    labels_to_use = labels[index:index + size]
    parametr = parametrs[i]
    plt.figure(figsize=(12, 8))
    for path, label in zip(files_to_process, labels_to_use):
        tradelog = pd.read_csv(path + 'stockApp_tradelog.csv')
        tradelog['timestamp'] = pd.to_datetime(tradelog['timestamp'])
        tradelog = tradelog[['timestamp', 'application_time']]
        start_time = tradelog['timestamp'].min()
        tradelog['relative_time'] = (tradelog['timestamp'] - start_time).dt.total_seconds()
        tradelog = tradelog.sort_values(by='relative_time')
        tradelog_resampled = tradelog.set_index('timestamp').resample('3s').mean().reset_index()
        tradelog_resampled = tradelog_resampled.dropna()
        tradelog_resampled.reset_index(inplace=True)
        plt.plot(tradelog_resampled['relative_time'], tradelog_resampled['application_time'], linestyle='-', label=label)
    plt.xlabel('Timestamp')
    plt.ylabel('Mean time (s)')
    plt.title('Mean transaction time | ' + parametr)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    file_name = f'{parametr}_plot.svg'
    #plt.savefig(os.path.join(output_path, file_name))
    plt.show()
    plt.close()
    index += size

# Cpu charts

In [ ]:
def process_file(file_path, container_id):
    file_path += 'stockApp_cpu.csv'
    data = pd.read_csv(file_path)
    filtered_data = data[data['contener_id'] == container_id]
    filtered_data = filtered_data[filtered_data['cpu_usage'] >= 3]
    filtered_data['timestamp'] = pd.to_datetime(filtered_data['timestamp'])
    start_time = filtered_data['timestamp'].min()
    filtered_data = filtered_data[['timestamp', 'cpu_usage']]
    filtered_data['relative_time'] = (filtered_data['timestamp'] - start_time).dt.total_seconds()
    filtered_data = filtered_data.set_index('timestamp').resample('15s').mean().reset_index()
    filtered_data = filtered_data.dropna()
    avg_cpu_usage = filtered_data['cpu_usage'].mean()
    return filtered_data, avg_cpu_usage

container_ids = ['stockproject-web-1', 'stockproject-db-1', 'stockproject-db_test-1', 'stockproject-locust-1', 'stockproject-celery_worker_execute_transactions-1']
output_paths = [
    'testCharts/users/cpu',
    'testCharts/requestTimes/cpu',
    'testCharts/transactionTime/cpu',
    'testCharts/numTrade/cpu',
    'testCharts/usersClass/cpu',
    'testCharts/none/cpu',
    'testCharts/usersAllClasses/cpu',
    'testCharts/requestTimesAllClasses/cpu',
    'testCharts/transactionTimeAllClasses/cpu',
    'testCharts/numTradeAllClasses/cpu',
    'testCharts/usersClassAllClasses/cpu'
]
for path in output_paths:
    os.makedirs(path, exist_ok=True)
group_sizes = [4,4,4,4,4]
index = 0

for i, size in enumerate(group_sizes):
    files_to_process = file_paths[index:index + size]
    labels_to_use = labels[index:index + size]
    output_path = output_paths[i]

    for container_id in container_ids:
        plt.figure(figsize=(12, 8))

        for file_path, label in zip(files_to_process, labels_to_use):
            data, avg_cpu_usage = process_file(file_path, container_id)
            print(f'Average usage | {label} | {container_id} : {avg_cpu_usage:.2f}%')
            plt.plot(data['relative_time'], data['cpu_usage'], linestyle='-', label=label)

        plt.xlabel('Time (s)')
        plt.ylabel('CPU usage (%)')
        plt.title(f'CPU usage | {container_id}')
        plt.grid(True)
        plt.legend()

        file_name = f'{container_id}_plot.svg'
        #plt.savefig(os.path.join(output_path, file_name))
        plt.show()
        plt.close()

    index += size

# RAM charts

In [ ]:
def process_file(file_path, container_id):
    file_path += 'stockApp_cpu.csv'
    data = pd.read_csv(file_path)
    filtered_data = data[data['contener_id'] == container_id]
    filtered_data = filtered_data[filtered_data['cpu_usage'] >= 0]
    filtered_data['timestamp'] = pd.to_datetime(filtered_data['timestamp'])
    start_time = filtered_data['timestamp'].min()
    filtered_data = filtered_data[['timestamp', 'memory_usage']]
    filtered_data['relative_time'] = (filtered_data['timestamp'] - start_time).dt.total_seconds()
    filtered_data = filtered_data.set_index('timestamp').resample('15s').mean().reset_index()
    filtered_data = filtered_data.dropna()
    return filtered_data
container_ids = ['stockproject-web-1', 'stockproject-db-1', 'stockproject-db_test-1', 'stockproject-locust-1', 'stockproject-celery_worker_execute_transactions-1']
output_paths = [
    'testCharts/users/RAM',
    'testCharts/requestTimes/RAM',
    'testCharts/transactionTime/RAM',
    'testCharts/numTrade/RAM',
    'testCharts/usersClass/RAM',
    'testCharts/none/RAM',
    'testCharts/usersAllClasses/RAM',
    'testCharts/requestTimesAllClasses/RAM',
    'testCharts/transactionTimeAllClasses/RAM',
    'testCharts/numTradeAllClasses/RAM',
]
for path in output_paths:
    os.makedirs(path, exist_ok=True)
group_sizes = [4,4,4,4,4]
index = 0

for i, size in enumerate(group_sizes):
    files_to_process = file_paths[index:index + size]
    labels_to_use = labels[index:index + size]
    output_path = output_paths[i]

    for container_id in container_ids:
        plt.figure(figsize=(12, 8))

        for file_path, label in zip(files_to_process, labels_to_use):
            data = process_file(file_path, container_id)
            plt.plot(data['relative_time'], data['memory_usage'], linestyle='-', label=label)
        plt.xlabel('Time (s)')
        plt.ylabel('RAM usage (MB)')
        plt.title(f'RAM usage | {container_id}')
        plt.grid(True)
        plt.legend()
        file_name = f'{container_id}_plot.svg'
        #plt.savefig(os.path.join(output_path, file_name))
        plt.show()
        plt.close()

    index += size